# Test OpenMeteo IFS forecast data integration

In [1]:
from dhis2eo.data.openmeteo.ecmwf_ifs import hourly
import s3fs
from datetime import date

In [2]:
fs = s3fs.S3FileSystem(anon=True)
var = 'temperature_2m'
day = date.fromisoformat('2020-01-01')
ds = hourly.open_ifs_file(
    fs=fs,
    day=day,
    variable=var,
    #chunks={'time': 2},
)
ds

<xarray.Dataset> Size: 232GB
Dimensions:  (dim0: 1, dim1: 6599680, dim2: 8784)
Dimensions without coordinates: dim0, dim1, dim2
Data variables:
             (dim0, dim1, dim2) float32 232GB dask.array<chunksize=(1, 6599680, 8784), meta=np.ndarray>

In [4]:
import s3fs
year = day.year
url = f"s3://openmeteo/data/ecmwf_ifs/{var}/year_{year}.om"
fs = s3fs.S3FileSystem(anon=True)
fsfile = fs.open(url)
fsfile

<File-like object S3FileSystem, openmeteo/data/ecmwf_ifs/temperature_2m/year_2020.om>

In [5]:
from omfiles import OmFileReader
omf = OmFileReader(fsfile)
omf

In [10]:
omf.shape

(1, 6599680, 8784)